<a href="https://colab.research.google.com/github/salarMokhtariL/House-Prices/blob/main/House_Prices_Predictions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Required Libraries

In [ ]:
!pip install pandas numpy torch torchvision scikit-learn matplotlib

# Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Loading the Data

In [ ]:
train_df = pd.read_csv('https://github.com/salarMokhtariL/House-Prices/raw/main/Dataset/train.csv')

test_df = pd.read_csv('https://github.com/salarMokhtariL/House-Prices/raw/main/Dataset/test.csv')

# Data Preprocessing

In [ ]:
# Drop the Id column
train_df.drop('Id', axis=1, inplace=True)
test_df.drop('Id', axis=1, inplace=True)

# Save the target variable
target = train_df['SalePrice']
train_df.drop('SalePrice', axis=1, inplace=True)

# Concatenate train and test data
all_data = pd.concat([train_df, test_df], axis=0)

# Select only numerical columns
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()
all_data = all_data[numerical_cols]

# Fill missing values with zeros
all_data.fillna(0, inplace=True)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
all_data = scaler.fit_transform(all_data)

# Split back into train and test data
train_data = all_data[:len(train_df)]
test_data = all_data[len(train_df):]

# Convert data to tensors
train_data = torch.FloatTensor(train_data)
target = torch.FloatTensor(target.values)
test_data = torch.FloatTensor(test_data)

# Creating the Model

In [ ]:
class HousePricesModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc3(out)
        return out

# Training the Model

In [ ]:
# Split train data into train and validation sets
train_data, val_data, target, val_target = train_test_split(train_data, target, test_size=0.2, random_state=42)

# Create datasets and data loaders
train_dataset = torch.utils.data.TensorDataset(train_data, target)
val_dataset = torch.utils.data.TensorDataset(val_data, val_target)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Create model and optimizer
model = HousePricesModel(train_data.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 200
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.shape[0]
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    model.eval()
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = criterion(output.view(-1), target)
            val_loss += loss.item() * data.shape[0]
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
    if epoch % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.show()

# Making Predictions on Test Data

In [ ]:
# Make predictions on test data
model.eval()
with torch.no_grad():
    predictions = model(test_data)

# Convert predictions to numpy array
predictions = predictions.view(-1).numpy()

# Save predictions to a CSV file
submission_df = pd.DataFrame({'Id': range(1461, 2920), 'SalePrice': predictions})
submission_df.to_csv('submission.csv', index=False)
